#### Pydantic basic: Creating and Using Models
Pydantic models are the foundation of data validation in Python. They use Python type annotations to define the structure and validate data at runtime. Here's detailed exploration of basic model creation with several examples.


In [1]:
from dataclasses import dataclass

@dataclass
class Person_data():
    name: str
    age: int
    city: str

person_data = Person_data(name="Alice", age=30, city="New York")
print(person_data)

Person_data(name='Alice', age=30, city='New York')


In [2]:
from pydantic import BaseModel

In [3]:
class Person_pydat(BaseModel):
    name: str
    age: int
    city: str

person_pydat = Person_pydat(name="Alice", age=30, city="New York")
print(person_pydat)

name='Alice' age=30 city='New York'


In [4]:
type(person_pydat)

__main__.Person_pydat

In [5]:
# Now suppose we made mistake in data entry, in place of string in 
# city we entered an integer, thereby violating the type constraint, 
# where as dataclass would accept it without any error.
Person_data_invalid = Person_data(name="Alice", age=30, city=123)
print(Person_data_invalid)

Person_data(name='Alice', age=30, city=123)


In [6]:
person_pydat_invalid = Person_pydat(name="Alice", age=30, city=12345)

ValidationError: 1 validation error for Person_pydat
city
  Input should be a valid string [type=string_type, input_value=12345, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type

#### 2. Model with Optional Field

Add optional field using Python's optional type:

In [8]:
from typing import Optional
class PersonPydanticOptional(BaseModel):
    name: str
    age: int
    city: Optional[str] = None
person_pydantic_optional = PersonPydanticOptional(name="Bob", age=25)
print(person_pydantic_optional)

name='Bob' age=25 city=None


In [14]:
from pydantic import BaseModel
from typing import List

class TeamPydantic(BaseModel):
    team_name: str
    members: List[str]
    team_size: int

In [15]:
TeamPydantic_lst=TeamPydantic(team_name="Developers", members=["Alice", "Bob", "Charlie"], team_size=3)

In [16]:
print(TeamPydantic_lst)

team_name='Developers' members=['Alice', 'Bob', 'Charlie'] team_size=3


In [18]:
# if you provide tuble instead of list for members, it still works
TeamPydantic_data=TeamPydantic(team_name="Developers", members=("Alice", "Bob", "Charlie"), team_size=3)

print(TeamPydantic_data)

team_name='Developers' members=['Alice', 'Bob', 'Charlie'] team_size=3


In [21]:
try:
    TeamPydantic_invalid=TeamPydantic(team_name="Developers", members="Alice, Bob, Charlie", team_size=3)
except ValueError as e:
    print("Error:", e)

Error: 1 validation error for TeamPydantic
members
  Input should be a valid list [type=list_type, input_value='Alice, Bob, Charlie', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/list_type


#### Pydantic Fields: Custom Validators and Constraints  

The field function in Pydantic enhances model fields beyound basic type hints by allowing you to specify 
validation rules, default values, aliases, and schema etc.

In [23]:
from pydantic import BaseModel, Field

In [24]:
class Item(BaseModel):
    name: str = Field(min_length=3, max_length=50)
    price: float = Field(gt=0, le =1000) # greater than 0 and less than or equal to 1000
    description: Optional[str] = Field(None, max_length=300)

In [25]:
try :
    Item_invalid = Item(name="AB", price=-10, description="This is a sample item description that exceeds the maximum length allowed for this field." * 10)
except ValueError as e:
    print("Validation Error:", e)

Validation Error: 3 validation errors for Item
name
  String should have at least 3 characters [type=string_too_short, input_value='AB', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_too_short
price
  Input should be greater than 0 [type=greater_than, input_value=-10, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than
description
  String should have at most 300 characters [type=string_too_long, input_value='This is a sample item de...allowed for this field.', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_too_long


In [27]:
try :
    Item_valid = Item(name="ABC", price=10, description="This is a sample item description that exceeds the maximum length allowed for this field.")
    print(Item_valid)
except ValueError as e:
    print("Validation Error:", e)

name='ABC' price=10.0 description='This is a sample item description that exceeds the maximum length allowed for this field.'


In [30]:
print(Item.model_json_schema())

{'properties': {'name': {'maxLength': 50, 'minLength': 3, 'title': 'Name', 'type': 'string'}, 'price': {'exclusiveMinimum': 0, 'maximum': 1000, 'title': 'Price', 'type': 'number'}, 'description': {'anyOf': [{'maxLength': 300, 'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Description'}}, 'required': ['name', 'price'], 'title': 'Item', 'type': 'object'}
